In [89]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [90]:
# 2020년대 트레이드 내역 데이터 불러오기
with open("player_moves/trade_list_2020s.json") as f:
    trades = json.load(f)

# 2020년데 지명권 트레이드 결과 데이터 불러오기
draft_tickets = pd.read_csv("player_moves/draft_tickets_2020s.csv")

In [91]:
draft_tickets.head(3)

,id,팀,지명라운드,선수,포지션
0,9,롯데,22-2-3,김세민,내야수
1,10,키움,22-2-4,노운현,투수
2,15,NC,22-2-4,조효원,내야수


In [92]:
url = "https://statiz.sporki.com/"
driver = webdriver.Chrome()
driver.get(url)

In [93]:
for trade in trades:
    players = trade["playerA"] + trade["playerB"]
    for player in players:
        if "억원" in player:
            continue
        elif "지명권" in player: # 지명권으로 실제 지명한 선수이름 추출
            try:
                search_player = \
                draft_tickets.loc[(draft_tickets["id"] == trade["id"]) & \
                    + (draft_tickets["지명라운드"] == player[:player.find("지명권")-1]), "선수"].item()
            except:
                continue
        else:
            search_player = player
        
        search_button = driver.find_element(By.CSS_SELECTOR, "body > div.warp > header > div.new_rightbox > div.new_searchPlayer > a")
        search_button.click()

        search_input = driver.find_element(By.CSS_SELECTOR, "#s")
        search_input.send_keys(search_player)

        enter_button = driver.find_element(By.CSS_SELECTOR, "body > div.warp > div.popBox.modal8 > form > div > input.btn")
        enter_button.click()

        time.sleep(1)

        statiz_player_id = None
        print(f"{search_player}")

        try: # 한번에 선수 기록 페이지로 접속 성공
            driver.find_element(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.player_info_header")
            statiz_player_id = driver.current_url[-5:]
        except: # 여러 행이 검색되거나 / 선수 데이터가 검색되지 않는 경우
            multiple_players = driver.find_elements(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.box_type_boared > div > table > tbody > tr")
            if len(multiple_players) == 0:
                continue
            elif len(multiple_players) == 1:
                if multiple_players[0].find_element(By.CSS_SELECTOR, "td").text == "검색된 선수가 없습니다.":
                    continue
                else: # 선수 목록 한 명만 뜨는데 바로 기록페이지로 접속되지 않는 경우 (예 : 김민식)
                    player_button = multiple_players[0].find_element(By.CSS_SELECTOR, "td > a")
                    player_button.click()
                    statiz_player_id = driver.current_url[-5:]
            else:
                player_candidat_list = [] # 동명이인 player code를 저장
                # 선수 한명 검색했을 때 여러명 이름 뜨는 경우 (예: 김상수, 김민)
                # 김상수 검색 -> 김상수(1990), 김상수(1988)
                # 김민 검색 -> 김민석, 김민재, 김민, 김민식, 김민수 ...
                # 정확히 검색어에 대응하는 이름만을 필터링한다 (김민 사례를 필터링)
                for idx, value in enumerate(multiple_players):
                    td = value.find_elements(By.CSS_SELECTOR, "td")
                    if td[0].text == search_player:
                        player_candidat_list.append(td[0].find_element(By.CSS_SELECTOR, "a").get_attribute("href")[-5:])
        else:
            pass
        finally:
            pass

김민식
김상슈
김상수
10320
10427
김민
13167
김도영
15056
12579
구자욱
장영석
박준태
11260
=1443
추재현
전병우
차재용
이흥련
김경호
11207
=1922
=1280
이승진
권기영
류지혁
홍건희
노수광
이태양
10681
10609
장현식
김태진
문경찬
박정수


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF7B48AFB05+28789]
	(No symbol) [0x00007FF7B48186E0]
	(No symbol) [0x00007FF7B46B592A]
	(No symbol) [0x00007FF7B468F505]
	(No symbol) [0x00007FF7B4736477]
	(No symbol) [0x00007FF7B474EF42]
	(No symbol) [0x00007FF7B472F1E3]
	(No symbol) [0x00007FF7B46FA938]
	(No symbol) [0x00007FF7B46FBAA1]
	GetHandleVerifier [0x00007FF7B4BE933D+3410093]
	GetHandleVerifier [0x00007FF7B4BFE7DD+3497293]
	GetHandleVerifier [0x00007FF7B4BF2A73+3448803]
	GetHandleVerifier [0x00007FF7B4977BBB+848171]
	(No symbol) [0x00007FF7B4823C3F]
	(No symbol) [0x00007FF7B481F6E4]
	(No symbol) [0x00007FF7B481F87D]
	(No symbol) [0x00007FF7B480ED49]
	BaseThreadInitThunk [0x00007FFE1FD77374+20]
	RtlUserThreadStart [0x00007FFE203DCC91+33]
